Step2 - semantically deduplicate all.csv after cleaning in excel and saving as tasks.csv

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util


# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-mpnet-base-v2')

/Users/alexc/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/alexc/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def remove_duplicate_tasks(csv_file):
    """
    Removes duplicate tasks from a CSV file using text embeddings.

    Args:
        csv_file: Path to the CSV file containing task data.

    Returns:
        A pandas DataFrame containing the filtered data with duplicate tasks removed.
    """

    df = pd.read_csv(csv_file)


    # Embed the 'Task' column using the model
    print("Embedding")
    task_embeddings = model.encode(df['Task'].tolist())

    # Calculate cosine similarities between all task embeddings
    similarities = util.cos_sim(task_embeddings, task_embeddings)

    # Set a threshold for similarity
    threshold = 0.85 

    # Create a mask to identify unique tasks
    unique_mask = []
    for i in range(len(similarities)):
        print(f"### {i+1}/{len(similarities)}")
        is_unique = True
        for j in range(len(similarities)):
            if i != j and similarities[i][j] > threshold:
                is_unique = False
                break
        unique_mask.append(is_unique)

    # Filter the DataFrame based on the unique mask
    filtered_df = df[unique_mask]

    return filtered_df

In [10]:
filtered_df.to_csv('tasks_filtered.csv')

In [6]:
# Load the CSV file
csv_file = "tasks.csv"

In [7]:
# Remove duplicate tasks
filtered_df = remove_duplicate_tasks(csv_file)

### 1/20594
### 2/20594
### 3/20594
### 4/20594
### 5/20594
### 6/20594
### 7/20594
### 8/20594
### 9/20594
### 10/20594
### 11/20594
### 12/20594
### 13/20594
### 14/20594
### 15/20594
### 16/20594
### 17/20594
### 18/20594
### 19/20594
### 20/20594
### 21/20594
### 22/20594
### 23/20594
### 24/20594
### 25/20594
### 26/20594
### 27/20594
### 28/20594
### 29/20594
### 30/20594
### 31/20594
### 32/20594
### 33/20594
### 34/20594
### 35/20594
### 36/20594
### 37/20594
### 38/20594
### 39/20594
### 40/20594
### 41/20594
### 42/20594
### 43/20594
### 44/20594
### 45/20594
### 46/20594
### 47/20594
### 48/20594
### 49/20594
### 50/20594
### 51/20594
### 52/20594
### 53/20594
### 54/20594
### 55/20594
### 56/20594
### 57/20594
### 58/20594
### 59/20594
### 60/20594
### 61/20594
### 62/20594
### 63/20594
### 64/20594
### 65/20594
### 66/20594
### 67/20594
### 68/20594
### 69/20594
### 70/20594
### 71/20594
### 72/20594
### 73/20594
### 74/20594
### 75/20594
### 76/20594
### 77/20594
### 78/2

In [ ]:
# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv("filtered_tasks.csv", index=False)

print("Duplicate tasks removed and saved to 'filtered_tasks.csv'.")

In [8]:
from sentence_transformers import CrossEncoder

# Load a pre-trained cross-encoder model
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def compute_similarity(task1, task2):
    # Prepare the input for the cross-encoder
    inputs = [[task1, task2]]

    # Compute similarity score
    scores = cross_encoder_model.predict(inputs)
    return scores[0]

# Test the function with example tasks
task1 = "User searches for 'Daniel Huynh' on X, views profile, and interacts with his posts."
task2 = "Go to X.com, search for Daniel Huynh, Like and repost his latest post."
task3 = "Create a new invoice in quickbooks"

print("Similarity between task1 and task2:", compute_similarity(task1, task2))
print("Similarity between task1 and task3:", compute_similarity(task1, task3))
print("Similarity between task2 and task3:", compute_similarity(task2, task3))


/Users/alexc/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Similarity between task1 and task2: 5.4367714
Similarity between task1 and task3: -10.517834
Similarity between task2 and task3: -10.468286


In [1]:
API_KEY = 'REDACTED'

# Imports

In [2]:
import pandas as pd
import os
import google.generativeai as genai
from datetime import datetime
import csv
import re

# Prompt

In [105]:
def get_prompt(JOB, CATEGORY):
    return f"""
    JOB
    {JOB}
    
    CATEGORY
    {CATEGORY}
    
    TABLE
    Index	Category	Task	App(s)	Example	App Count	Time Estimate (seconds)	Difficulty (1-10)
    1	Administration	Add a new contact to your phone	Contacts, Google Contacts, iCloud Contacts	Add a new contact named "John Smith" to your phone's contacts, including his phone number and email.	3	30	2
    2	Administration	Create a reminder in your calendar	Google Calendar, Outlook Calendar, Apple Calendar	Set a reminder for yourself to call your mom at 6:00 PM tonight.	3	20	1
    3	Administration	Create a to-do list in a task manager	Google Tasks, Todoist, TickTick, Microsoft To Do	Add a task to your to-do list: "Buy groceries for the week."	4	15	1
    4	Administration	Set up a recurring appointment in your calendar	Google Calendar, Outlook Calendar, Apple Calendar	Schedule a weekly yoga class on Tuesdays at 6:00 PM.	3	30	2
    5	Administration	Change your password for a website	Account settings of any website	Change your password for your online banking account, making sure it's strong and unique.	1	45	3
    6	Communication	Send a quick text message	Messages, WhatsApp, Telegram, Facebook Messenger	Send a quick message to your friend: "Hey, are you free tonight?"	4	10	1
    7	Communication	Send a quick email	Gmail, Outlook, Yahoo Mail, Apple Mail	Send a quick email to your friend inviting them to grab coffee tomorrow.	4	20	2
    8	Communication	Share a photo	Instagram, Facebook, WhatsApp, Telegram	Share a picture of your lunch on your Instagram story.	4	15	1
    9	Communication	Send a voice message	Voicemail, WhatsApp Voice Messages, Telegram Voice Messages	Leave a voice message for your friend letting them know you'll be late.	3	30	2
    10	Communication	Join a group chat	WhatsApp, Telegram, Facebook Messenger, Slack	Join a group chat invitation to coordinate the next family gathering.	4	10	1
    11	Communication	Update your profile picture	Facebook, Instagram, Twitter, LinkedIn	Change your profile picture on Facebook to a new photo of you.	4	20	2
    12	Creative Work	Create a simple drawing	Adobe Photoshop, Autodesk Sketchbook, Procreate	Draw a quick doodle in Adobe Photoshop using a simple brush.	3	45	3
    13	Creative Work	Edit a photo	Adobe Photoshop, GIMP, Snapseed, VSCO	Adjust the brightness and contrast of a photo in Snapseed.	4	20	2
    14	Creative Work	Create a simple graphic	Canva, Adobe Illustrator, Figma, Affinity Designer	Create a quick social media post graphic in Canva using a template.	4	30	2
    15	Creative Work	Add text to a photo	Adobe Photoshop, GIMP	Add a text overlay to a photo using Adobe Photoshop.	2	30	3
    16	Creative Work	Edit a video clip	Adobe Premiere Pro, Final Cut Pro, DaVinci Resolve	Trim the beginning and end of a video clip to remove unwanted parts.	3	45	4
    17	Customer Service	Ask a question on a company's website live chat	Intercom, LiveChat, Drift, Tawk.to, Olark	Ask a question about shipping times for your recent order using the company's live chat.	5	30	2
    18	Data Management	Enter data into a spreadsheet	Google Sheets, Microsoft Excel	Input your grocery expenses for the week into a spreadsheet.	2	45	3
    19	Data Management	Create a simple chart	Google Sheets, Microsoft Excel	Create a pie chart in Google Sheets to show your monthly budget allocation.	2	30	3
    20	Development	Run a code snippet	Jupyter Notebook, Google Colab	Run a basic Python code snippet to print "Hello, World!"	2	15	2
    21	Development	Edit a code file	Visual Studio Code, Atom	Correct a simple error in a Python code file using Visual Studio Code.	2	60	4
    22	Development	Save changes to a code file	Visual Studio Code, Atom	Save changes to a code file in Visual Studio Code after making a correction.	2	5	1
    23	Entertainment	Listen to a song	Spotify, Apple Music, YouTube Music	Play your favorite song on Spotify.	3	10	1
    24	Entertainment	Watch a short video	YouTube, Vimeo	Watch a short video on YouTube about a trending topic.	2	30	1
    25	Entertainment	Play a mobile game	Android Play Store, Apple App Store	Play a quick round of a mobile game on your phone.	2	10	1
    26	Finance	Check your bank account balance	Online banking app for your bank	Check your account balance using your bank's online banking app.	1	15	1
    27	Finance	Track an expense	Mint, YNAB, Personal Capital, EveryDollar, PocketGuard	Log a recent purchase of coffee using your budgeting app.	5	10	1
    28	Finance	Transfer money between accounts	Online banking app for your bank	Transfer $10 from your checking account to your savings account.	1	20	2
    29	Health and Fitness	Track a workout	Strava, MyFitnessPal, Fitbit, Apple Health	Log a 20-minute run in your fitness app, including distance and time.	4	30	2
    30	Health and Fitness	Check your sleep data	Fitbit, Apple Health	Check your sleep time and quality data using your Fitbit.	2	15	1
    31	Health and Fitness	Log your food intake	MyFitnessPal, Lose It, Cronometer	Log your breakfast today in your nutrition tracking app: "Oatmeal with fruit."	3	15	2
    32	Learning	Start a lesson	Duolingo, Babbel, Khan Academy	Start a lesson in Duolingo to learn Spanish.	3	20	2
    33	Learning	Watch a YouTube tutorial	YouTube	Watch a short YouTube tutorial on a new cooking technique.	1	30	1
    34	Learning	Search for a book	Amazon, Barnes & Noble, Google Books	Search for a book on a specific topic on Amazon and add it to your cart.	3	20	2
    35	Marketing	Schedule a social media post	Hootsuite, Buffer, SproutSocial, Later, TweetDeck	Schedule a post on your Instagram account to share a photo with a caption.	5	30	3
    36	Marketing	Create a simple infographic	Canva	Create a simple infographic in Canva to visually represent a statistic related to your business.	1	60	4
    37	Marketing	Check your website analytics	Google Analytics	Check the number of visitors to your website today using Google Analytics.	1	15	1
    38	Marketing	Reply to a comment on your social media post	Facebook, Instagram, Twitter	Reply to a comment on your latest tweet, thanking the commenter.	3	10	1
    39	Media Editing	Trim an audio recording	Audacity, GarageBand, Adobe Audition	Trim the beginning and end of a voice recording to remove silence.	3	30	3
    40	Media Editing	Convert an image to a different format	GIMP, ImageMagick, XnConvert, FreeImage, IrfanView	Convert a JPG image to a PNG format.	5	20	2
    41	Media Editing	Adjust the brightness of a photo	Adobe Photoshop, GIMP, Snapseed, VSCO	Adjust the brightness of a photo to make it more appealing.	4	15	2
    42	Media Editing	Add a simple transition effect to a video	Adobe Premiere Pro, Final Cut Pro, DaVinci Resolve	Add a simple fade-in transition effect to a video clip.	3	45	4
    43	Shopping	Add an item to your shopping cart	Amazon, eBay, Etsy	Add a new pair of shoes to your Amazon shopping cart.	3	10	1
    44	Shopping	Read product reviews	Amazon, eBay, Etsy	Read product reviews for a new laptop you're considering buying on Amazon.	3	60	3
    45	System Settings	Check your computer's storage space	Storage settings in OS	Check how much storage space is available on your computer.	1	10	1
    46	System Settings	Adjust your screen brightness	Display settings in OS	Adjust your computer's screen brightness to a comfortable level.	1	5	1
    47	Travel and Booking	Search for a flight	Expedia, Kayak, Skyscanner, Google Flights, Hopper	Search for a flight from New York to Los Angeles on Expedia.	5	30	2
    48	Travel and Booking	Check hotel prices	Booking.com, Airbnb, Expedia, Hotels.com	Check the prices for a hotel in Paris for your upcoming trip.	4	30	2
    49	Travel and Booking	Look up driving directions	Google Maps, Apple Maps, Waze	Look up driving directions from your current location to the grocery store.	3	15	1
    50	Crypto	Check your crypto wallet balance	MetaMask, Coinbase Wallet, Trust Wallet	Check the balance of your cryptocurrency portfolio in your MetaMask wallet.	3	15	2
    51	Crypto	Track the price of a cryptocurrency	CoinMarketCap, CoinGecko, TradingView	Track the current price of Bitcoin on CoinMarketCap.	3	10	1
    52	Home Automation	Turn on your smart lights	Google Home, Alexa, HomeKit	Turn on the lights in your living room using the Google Home app.	3	5	1
    53	Home Automation	Adjust your thermostat	Google Home, Alexa, HomeKit	Increase your thermostat by 2 degrees using the Alexa app.	3	10	1
    54	Home Automation	Play music	Google Home, Alexa, Sonos	Play your favorite song on your Google Home speaker.	3	10	1
    55	Food & Beverages	Order food delivery	Uber Eats, DoorDash, Grubhub	Order a salad from your favorite restaurant using the Uber Eats app.	3	20	2
    56	Food & Beverages	Search for a recipe	Allrecipes, Food Network, Epicurious	Find a quick and easy dinner recipe on Allrecipes.	3	20	2
    57	Social Media	Post a status update	Facebook, Instagram, Twitter, LinkedIn	Share a quick status update on Facebook about your day.	4	15	2
    58	Social Media	Like a friend's post	Facebook, Instagram, Twitter, LinkedIn	Like a friend's photo on Instagram.	4	5	1
    59	Social Media	Reply to a comment	Facebook, Instagram, Twitter, LinkedIn	Reply to a comment on your latest tweet, thanking the commenter.	4	10	1
    60	Education & Research	Look up the definition of a word	Merriam-Webster, Dictionary.com, Oxford Dictionaries	Look up the definition of the word "ubiquitous" on Merriam-Webster.	3	15	1
    61	Entertainment & Media	Watch a short YouTube video	YouTube, Vimeo	Watch a short video on YouTube about a funny cat.	2	20	1
    62	Personal Care	Book an appointment	Online booking systems for salons, doctors, etc.	Book a haircut appointment for next week at your favorite salon.	1	30	2
    63	Personal Care	Search for a local service	Yelp, Google Maps	Search for a local coffee shop on Yelp.	2	20	2
    64	Shopping & Retail	Add an item to your wishlist	Amazon, eBay, Etsy	Add a new book to your Amazon wishlist.	3	10	1
    65	Finance & Budgeting	Check your credit card balance	Online account for your credit card provider	Check the balance of your credit card online.	1	15	1
    66	Art Direction	Create a simple mockup using Canva	Canva	Create a simple mockup of a social media post using Canva.	1	45	3
    67	Art Direction	Find a free stock photo	Unsplash, Pixabay, Pexels	Find a free stock photo on Unsplash of a food item.	3	15	1
    68	Accounting	Record a single transaction in accounting software	Xero, QuickBooks, FreshBooks	Record a payment received from a client in Xero.	3	20	2
    69	Accounting	View a basic report	Xero, QuickBooks, FreshBooks	View a simple report in QuickBooks showing the current balance of your client's account.	3	15	1
    70	Acting	Search for an audition opportunity	Backstage, Actors Access	Search for an audition for a short film on Backstage.	2	30	2
    71	Acting	Read a movie script online	Script databases (e.g., IMSDb, Simply Scripts)	Read a short film script online.	2	45	3
    72	Architecture	Use a 3D modeling tool to create a simple model	SketchUp, Blender, Autodesk Revit	Create a simple 3D model of a chair in SketchUp.	3	60	4
    73	Architecture	Search for architectural styles	Pinterest, Houzz	Search for examples of Victorian architecture on Pinterest.	2	20	2
    74	Bioinformatics	View a protein sequence	UniProt	View the amino acid sequence of a specific protein in UniProt.	1	15	2
    75	Broadcast News	Use an audio editing tool to adjust volume	Audacity, Adobe Audition	Adjust the volume of an audio recording in Audacity.	2	30	3
    76	Broadcast News	Write a short social media post for news	Facebook, Twitter	Write a short social media post for a breaking news story.	2	20	2
    77	Chemical Science	Search for a chemical compound	PubChem	Search for the chemical compound "carbon dioxide" on PubChem.	1	15	2
    78	Civil Engineering	Use a CAD software to draw a simple shape	AutoCAD, Revit	Draw a circle using AutoCAD.	2	15	2
    79	Civil Engineering	View a construction site plan	AutoCAD, Revit	View a construction site plan for a new apartment building.	2	30	2
    80	Computer Science	Run a simple Python script	Visual Studio Code, Atom	Run a Python script to calculate the average of three numbers.	2	20	3
    81	Computer Science	Search for a programming language tutorial	Codecademy, W3Schools, FreeCodeCamp	Search for a beginner's Python tutorial on Codecademy.	3	20	2
    82	Computer Science	Open the Developer Console in a browser	Chrome DevTools, Firefox Developer Tools, Edge Developer Tools	Open the Developer Console in Chrome to view the code of a webpage.	3	5	1
    83	Dance	Watch a dance tutorial video on YouTube	YouTube	Watch a short dance tutorial for a specific move on YouTube.	1	30	1
    84	Dance	Share a dance video on TikTok	TikTok	Share a short dance video on your TikTok profile.	1	20	2
    85	Dance	Search for dance studios	Google Maps, Yelp	Search for dance studios in your city on Google Maps.	2	20	2
    86	Project Management	Create a new project in a project management tool	Trello, Asana, Jira	Create a new project in Trello to manage the launch of a new product.	3	30	2
    87	Project Management	Assign tasks to team members	Trello, Asana, Jira	Assign a specific task to a team member in Asana with a deadline.	3	15	1
    88	Project Management	Track project progress	Trello, Asana, Jira	View the progress of your project in Trello, including completed tasks and upcoming deadlines.	3	20	2
    89	Project Management	Collaborate on a project with team members	Trello, Asana, Jira	Discuss project updates and make changes to tasks in Asana with your team.	3	45	3
    90	Research & Information	Search for academic articles	Google Scholar, PubMed, JSTOR	Search for academic articles on the topic of climate change using Google Scholar.	3	30	2
    91	Research & Information	Read and analyze research papers	PDF readers (Adobe Acrobat Reader, etc.)	Read and analyze a research paper on a specific topic using Adobe Acrobat Reader.	1	60	4
    92	Research & Information	Create citations and bibliographies	Zotero, Mendeley, EndNote	Create a bibliography for a research paper using Zotero.	3	30	3
    93	Research & Information	Search for news articles	Google News, BBC News, CNN	Search for news articles about the latest political developments using Google News.	3	20	2
    94	Design & Graphics	Create a simple logo	Canva, Adobe Illustrator, Figma	Create a simple logo for your personal blog using Canva.	3	60	4
    95	Design & Graphics	Create a website mockup	Figma, Adobe XD, Sketch	Create a mockup of a website layout using Figma.	3	60	4
    96	Design & Graphics	Edit a vector graphic	Adobe Illustrator, Inkscape	Edit a vector graphic of a company logo in Adobe Illustrator.	2	45	3
    97	Design & Graphics	Create a simple infographic	Canva, Adobe Spark, Piktochart	Create a simple infographic to visualize data about your company's growth using Canva.	3	45	3
    98	Video Games	Play a video game	Steam, Epic Games Store, Origin	Play a game of "Fortnite" on your computer using Epic Games Store.	3	10	1
    99	Video Games	Stream gameplay	Twitch, YouTube Gaming	Stream your gameplay of "Minecraft" on Twitch.	2	30	2
    100	Video Games	Create a game mod	Modding tools for specific games	Create a new weapon mod for the game "Skyrim" using the Nexus Mod Manager.	1	120	6
    101	Programming	Write a simple Python script	Visual Studio Code, Atom	Write a Python script to check if a number is even or odd.	2	30	3
    102	Programming	Debug a code snippet	Visual Studio Code, Atom	Debug a Python script to find and fix an error in the code.	2	45	4
    103	Programming	Run unit tests	Python unittest, pytest	Run unit tests to ensure that a Python function is working correctly.	2	20	3
    104	Programming	Use a version control system	Git	Commit changes to a Python code project using Git and push them to a remote repository.	1	30	3
    105	Software Development	Design a user interface	Figma, Adobe XD, Sketch	Design the user interface for a new mobile app using Figma.	3	60	4
    106	Software Development	Develop a web application	HTML, CSS, JavaScript, Python (with frameworks like Django or Flask)	Develop a simple web application using HTML, CSS, and JavaScript to display a list of products.	4	120	6
    107	Software Development	Deploy a web application	Heroku, AWS, Google Cloud Platform	Deploy a simple web application to Heroku.	3	60	4
    108	Education & Training	Take an online course	Coursera, Udemy, edX	Enroll in a course on data science on Coursera and complete a lesson.	3	30	2
    109	Education & Training	Attend a webinar	Zoom, GoToWebinar	Attend a webinar on digital marketing and take notes.	2	60	3
    110	Education & Training	Create educational materials	PowerPoint, Google Slides, Prezi	Create a presentation for a training session on a new software tool.	3	60	4
    111	Data Analysis	Clean data in a spreadsheet	Google Sheets, Microsoft Excel	Clean up messy data in a spreadsheet by removing duplicates and fixing inconsistencies.	2	45	3
    112	Data Analysis	Create a visualization	Google Sheets, Microsoft Excel, Tableau	Create a bar chart in Google Sheets to visualize sales data for the last year.	3	30	3
    113	Data Analysis	Build a statistical model	Python (with libraries like scikit-learn)	Build a simple linear regression model in Python to predict sales based on marketing spend.	1	90	6
    114	Data Analysis	Analyze data using SQL	SQL databases (MySQL, PostgreSQL, etc.)	Use SQL to query a database and analyze sales data for a specific product.	1	30	3
    115	Security & Privacy	Manage passwords	Password managers (LastPass, 1Password, Dashlane)	Store and manage your passwords using LastPass.	3	20	2
    116	Security & Privacy	Use a VPN	NordVPN, ExpressVPN, CyberGhost	Connect to a VPN server to encrypt your internet traffic and protect your privacy.	3	15	2
    117	Security & Privacy	Scan your computer for malware	Antivirus software (Norton, McAfee, AVG)	Run a full system scan for malware using Norton Antivirus.	1	60	2
    118	Security & Privacy	Set up two-factor authentication	Two-factor authentication settings for accounts	Set up two-factor authentication for your Google account.	1	15	2
    119	Automation & Scripting	Automate a repetitive task	AutoHotkey, Keyboard Maestro, AppleScript	Create an AutoHotkey script to automate a repetitive task, such as sending emails.	1	60	4
    120	Automation & Scripting	Use a macro recorder	Macro recording software (AutoHotkey, Keyboard Maestro)	Record a series of actions in a macro recorder to automate a task.	1	30	3
    121	Automation & Scripting	Use a scripting language	Python, JavaScript, Bash	Write a simple Python script to download files from a website.	1	30	3
    122	Writing	Write a blog post	WordPress, Medium, Blogger	Write a blog post about a recent travel experience using WordPress.	3	60	3
    123	Writing	Write an email	Gmail, Outlook	Compose a professional email to a client.	2	30	2
    124	Writing	Write a letter	Microsoft Word, Google Docs	Write a formal letter to a government agency.	2	45	3
    125	Writing	Use a grammar checker	Grammarly, ProWritingAid, Hemingway Editor	Correct grammar and style errors in a document using Grammarly.	3	30	2
    126	Writing	Use a plagiarism checker	Plagiarism checkers (Grammarly, Turnitin, etc.)	Check a document for plagiarism using Grammarly.	1	15	1
    127	Web Development	Create a website using a website builder	Wix, Squarespace, WordPress	Create a simple website using Wix to showcase your portfolio.	3	60	3
    128	Web Development	Use HTML, CSS, and JavaScript	Text editors (Visual Studio Code, Atom), Web browsers (Chrome, Firefox)	Create a simple HTML webpage that displays a heading and paragraph.	3	30	2
    129	Web Development	Use a framework like React or Angular	JavaScript frameworks (React, Angular)	Build a dynamic web application using React to display a list of products.	2	120	6
    130	Web Development	Test a webpage	Browser developer tools, Selenium	Test a webpage using the developer tools in Chrome to ensure that it works correctly on different devices and browsers.	2	45	3
    131	Web Development	Deploy a website	Hosting providers (GoDaddy, Bluehost, HostGator), Cloud platforms (AWS, Google Cloud)	Deploy a website to GoDaddy using FTP.	2	45	3
    132	Photography	Edit photos in a photo editor	Adobe Photoshop, GIMP, Luminar	Edit a photo in Adobe Photoshop to adjust the colors, contrast, and brightness.	3	60	4
    133	Photography	Use a photo editing app	Snapseed, VSCO	Edit a photo in Snapseed to apply filters and make basic adjustments.	2	20	2
    134	Photography	Capture a photo	Camera software (Canon, Nikon, Sony)	Capture a photo using a DSLR camera and a specific lens.	1	30	2
    135	Photography	Share photos on social media	Facebook, Instagram, Twitter	Share a photo of your trip on Instagram.	3	15	1
    136	Video Editing	Create a video using a video editor	Adobe Premiere Pro, Final Cut Pro, DaVinci Resolve	Create a short video using Adobe Premiere Pro, including editing clips, adding transitions, and adding music.	3	120	6
    137	Video Editing	Use a video editing app	iMovie, KineMaster	Edit a short video clip in iMovie to add music and titles.	2	30	3
    138	Video Editing	Export a video in a specific format	Video editing software	Export a video in a format suitable for YouTube.	1	30	2
    139	Video Editing	Share a video on social media	YouTube, Facebook, Instagram	Upload a video of your trip to YouTube.	3	30	2
    140	Music	Create a song	Music production software (Ableton Live, FL Studio)	Create a simple song using Ableton Live, including adding instruments, recording vocals, and mixing.	1	120	6
    141	Music	Listen to music	Spotify, Apple Music, YouTube Music	Listen to your favorite playlist on Spotify.	3	10	1
    142	Music	Create a playlist	Spotify, Apple Music, YouTube Music	Create a playlist of your favorite songs on Spotify.	3	20	2
    143	Music	Share a song or playlist	Spotify, Apple Music, YouTube Music	Share a playlist of your favorite songs with a friend on Spotify.	3	10	1
    144	Animation	Create a simple animation	Adobe After Effects, Blender	Create a simple animation of a bouncing ball using Adobe After Effects.	2	60	4
    145	Animation	Use an animation software	Toon Boom Harmony, Adobe Animate	Create a simple animation of a character walking using Toon Boom Harmony.	2	60	4
    146	Animation	Export an animation in a specific format	Animation software	Export an animation in a format suitable for web playback.	1	30	2
    147	Animation	Share an animation online	YouTube, Vimeo, social media	Upload an animation to YouTube.	3	20	2
    148	Graphic Design	Create a simple flyer	Canva, Adobe InDesign	Design a simple flyer for an upcoming event using Canva.	2	45	3
    149	Graphic Design	Use a graphic design app	Canva, Adobe Spark	Create a simple graphic for a social media post using Canva.	2	30	2
    150	Graphic Design	Export a graphic in a specific format	Graphic design software	Export a flyer in a format suitable for printing.	1	20	2
    
    INSTRUCTION
    generate a table called NEW_TABLE, an exhaustive and comprehensive list of tasks that would be done on a computer for the given JOB={JOB} in a given work CATEGORY={CATEGORY}
    return NEW_TABLE in the same format as the sample TABLE
    make the examples specific - i need to be able to give this table to many actuaries and get them to do screen recordings of themselves doing the examples. it cannot be too complicated or vague.
    
    Here is a bad example - it's bad because it is asking too much of the user and is asking the user to do multiple things for the example and there's no way this could be done in 60 seconds: 
    31  Finance Use a pricing software  Specialized actuarial software  Actuarial Edge  Price a new insurance product using Actuarial Edge, considering factors like mortality rates, interest rates, and expenses. 1 60  4
    
    Here is a good example - it's good because it can be done quickly and is a single task and it's the right level of detail:
    57 Social Media Post a status update Facebook, Instagram, Twitter, LinkedIn Share a quick status update on Facebook about your day. 4 15 2
    
    lean towards over-estimating the time and difficulty. make the list of apps exhaustive and comprehensive for someone doing JOB on a computer. 
    
    a reasonable average person should agree with the time and difficulty estimates.
    
    make the examples as specific as possible without revealing personal information. 
    
    make your output contain only the required output NEW_TABLE giving examples of tasks done by people in the role of {JOB} ({CATEGORY}). 
    
    NEW_TABLE should have the following columns in order only: 
    Index,Category,Task,App(s),Example,App Count,Time Estimate (seconds),Difficulty (1-10)

    NEW_TABLE should only contain an exhaustive list of tasks done on a computer by a {JOB}.
    
    return nothing else but NEW_TABLE in the same format as TABLE with "|" delimiters.
    """

# Setup Gemini

In [106]:
genai.configure(api_key=API_KEY)

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
]

# Load Jobs

In [107]:
# Load the CSV file
file_path = 'JOBS.csv'
jobs = pd.read_csv(file_path)

print(jobs.head())
print("...")
print(jobs.tail())

   Index                       Job                 Category
0      1           Account Manager    Business & Management
1      2                Accountant                  Finance
2      3                   Actuary                  Finance
3      4         Actuary Assistant                  Finance
4      5  Administrative Assistant  Office & Administration
...
     Index                              Job         Category
495    496                        Counselor  Social Services
496    497               Counselor (Career)  Social Services
497    498    Counselor (Marriage & Family)  Social Services
498    499               Counselor (School)        Education
499    500  Customer Service Representative            Sales


# Use gemini to get tasks for each job

In [108]:
for index, row in jobs.iterrows():
    JOB = row['Job']
    CATEGORY = row['Category']

    # output csv of tasks for this job
    FILENAME = f'JOB_TASKS/{index}_{JOB}_{CATEGORY}.csv'

    PROMPT = get_prompt(JOB, CATEGORY)

    #print(PROMPT)

    # skip if already done
    if os.path.exists(FILENAME):
        print(f"[loading saved data] {index+1}/{len(jobs)} Job: {JOB}, Category: {CATEGORY}")
    else:
        print(f"{index+1}/{len(jobs)} Job: {JOB}, Category: {CATEGORY}")

        try:
            model = None
            
            # get jobs CSV using gemini
            model = genai.GenerativeModel(
                model_name="gemini-1.5-flash",
                safety_settings=safety_settings,
                generation_config=generation_config,
            )
            
            start_time = datetime.now()
            print(f"Start time: {start_time}")
            
            chat_session = model.start_chat(
                history=[
                    {
                        "role": "user",
                        "parts": [
                            PROMPT,
                        ],
                    }
                ]
            )
            
            response = chat_session.send_message("Produce NEW_TABLE as per the instructions above.")
            end_time = datetime.now()
            duration = end_time - start_time
            
            print(f"End time: {end_time}")
            print(f"Gemini API request duration: {duration}")
        
            # Remove extra newlines and split the response text into lines
            lines = response.text.strip().split('\n')[0:]
            
            # Extract the header and rows
            header = re.split(r'\s*\|\s*', lines[0].strip('|'))
            rows = [re.split(r'\s*\|\s*', line.strip('|')) for line in lines[1:]]
            
            # Write to CSV
            with open(FILENAME, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(header)
                csvwriter.writerows(rows)
        
            print("Saving file")
        except Exception as e:
            print(f"[ERROR] {index+1}/{len(jobs)} Job: {JOB}, Category: {CATEGORY}")
            print(e)

[loading saved data] 1/500 Job: Account Manager, Category: Business & Management
[loading saved data] 2/500 Job: Accountant, Category: Finance
[loading saved data] 3/500 Job: Actuary, Category: Finance
[loading saved data] 4/500 Job: Actuary Assistant, Category: Finance
[loading saved data] 5/500 Job: Administrative Assistant, Category: Office & Administration
[loading saved data] 6/500 Job: Advertising Executive, Category: Media and Communication
[loading saved data] 7/500 Job: Aerospace Engineer, Category: Engineering
[loading saved data] 8/500 Job: Agricultural Engineer, Category: Engineering
[loading saved data] 9/500 Job: Air Traffic Controller, Category: Transportation
[loading saved data] 10/500 Job: Analyst (Business, Financial, Market Research), Category: Business & Finance
[loading saved data] 11/500 Job: Animator, Category: Arts and Design
[loading saved data] 12/500 Job: Animator (2D), Category: Arts and Design
[loading saved data] 13/500 Job: Animator (3D), Category: Arts 

In [109]:
1/0

ZeroDivisionError: division by zero